<a href="https://colab.research.google.com/github/talhaanwarch/ODIR2019/blob/master/single_eye%20training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:

#connect drive with colab notebook/
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:

!pip install -U efficientnet==0.0.4
!pip install keras-rectified-adam

  Created wheel for efficientnet: filename=efficientnet-0.0.4-cp36-none-any.whl size=14288 sha256=dae184209867a2a65a8ded6c5329967a478a903dcf1a000864232876ceb3e8bc
  Stored in directory: /root/.cache/pip/wheels/5c/34/68/a611a699a28239e964ccf144c0e767cdb5439fee82ec5de6e0
Successfully built efficientnet
  Created wheel for keras-rectified-adam: filename=keras_rectified_adam-0.17.0-cp36-none-any.whl size=14781 sha256=01659e0c85a23a7bbf1220f86ef6eb163cac43c5d923476513c045ba80a756df
  Stored in directory: /root/.cache/pip/wheels/7b/01/27/3a934e1a5644f5b93c720422a6ef97034ea78a21ba71cfb549
Successfully built keras-rectified-adam


In [0]:

#following are libraries to unpack 7z file
!pip install patool 
!pip install pyunpack
from pyunpack import Archive

     |████████████████████████████████| 81kB 3.1MB/s 
  Created wheel for pyunpack: filename=pyunpack-0.1.2-cp36-none-any.whl size=5163 sha256=22d9c48b2f0deb5d1f2234acba0534776ede82c6e4f34aa16dc91725fda966cb
  Stored in directory: /root/.cache/pip/wheels/af/44/08/60613970881e542c0baad1f2dea5ed8e6716bc573f49197b7e
Successfully built pyunpack


In [0]:
!mkdir data


In [0]:
import pandas as pd
import numpy as np
import cv2

# Prepare data

In [0]:
path='/content/drive/My Drive/thesis/'


In [0]:

#extract the 7z file in data folder we created above
#this not only extract the data but move it from drive to colab for efficinet processing
#it will take some time
Archive(path+'ODIR-5K_Training_Images.7z').extractall('data')

In [0]:
Archive(path+'ODIR-5K_Testing_Images.7z').extractall('data')


In [0]:
#read the annottion file
annotation=pd.read_excel(path+'ODIR-5K_Training_Annotations(Updated)_V2.xlsx')

In [0]:
annotation.head()


,ID,Patient Age,Patient Sex,Left-Fundus,Right-Fundus,Left-Diagnostic Keywords,Right-Diagnostic Keywords,N,D,G,C,A,H,M,O
0,0,69,Female,0_left.jpg,0_right.jpg,cataract,normal fundus,0,0,0,1,0,0,0,0
1,1,57,Male,1_left.jpg,1_right.jpg,normal fundus,normal fundus,1,0,0,0,0,0,0,0
2,2,42,Male,2_left.jpg,2_right.jpg,laser spot，moderate non proliferative retinopathy,moderate non proliferative retinopathy,0,1,0,0,0,0,0,1
3,3,66,Male,3_left.jpg,3_right.jpg,normal fundus,branch retinal artery occlusion,0,0,0,0,0,0,0,1
4,4,53,Male,4_left.jpg,4_right.jpg,macular epiretinal membrane,mild nonproliferative retinopathy,0,1,0,0,0,0,0,1


In [0]:
left_train=pd.read_csv(path+'left_annotation.csv')
left_train.drop('Unnamed: 0',inplace=True,axis=1)
right_train=pd.read_csv(path+'right_annotation.csv')
right_train.drop('Unnamed: 0',inplace=True,axis=1)


In [0]:
left_train.head()

,Image,Label,N,D,G,C,A,H,M,O
0,0_left.jpg,C,0,0,0,1,0,0,0,0
1,1_left.jpg,N,1,0,0,0,0,0,0,0
2,2_left.jpg,O,0,0,0,0,0,0,0,1
3,3_left.jpg,N,1,0,0,0,0,0,0,0
4,4_left.jpg,O,0,0,0,0,0,0,0,1


In [0]:
train=pd.concat([left_train,right_train])

In [0]:
from sklearn.model_selection import train_test_split
df_train, df_validation = train_test_split(train, test_size = 0.2, stratify=train[['N','D','G','C','A','H','M','O']], random_state = 2020)

In [0]:
df_train.head()

,Image,Label,N,D,G,C,A,H,M,O
3200,4343_right.jpg,D,0,1,0,0,0,0,0,0
2069,2692_left.jpg,N,1,0,0,0,0,0,0,0
1677,2220_left.jpg,O,0,0,0,0,0,0,0,1
2240,2863_right.jpg,O,0,0,0,0,0,0,0,1
578,579_right.jpg,O,0,0,0,0,0,0,0,1


In [0]:
df_validation.head()

,Image,Label,N,D,G,C,A,H,M,O
3283,4437_left.jpg,D,0,1,0,0,0,0,0,0
1604,2143_right.jpg,C,0,0,0,1,0,0,0,0
1869,2492_left.jpg,N,1,0,0,0,0,0,0,0
2277,2900_right.jpg,N,1,0,0,0,0,0,0,0
3361,4530_left.jpg,O,0,0,0,0,0,0,0,1


# Utility Functions

## Image Preprocessing

In [0]:
#https://www.kaggle.com/ratthachat/aptos-eye-preprocessing-in-diabetic-retinopathy
def crop_image1(img,tol=7):
    # img is image data
    # tol  is tolerance
        
    mask = img>tol
    return img[np.ix_(mask.any(1),mask.any(0))]

def crop_image_from_gray(img,tol=7):
    if img.ndim ==2:
        mask = img>tol
        return img[np.ix_(mask.any(1),mask.any(0))]
    elif img.ndim==3:
        gray_img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
        mask = gray_img>tol
        
        check_shape = img[:,:,0][np.ix_(mask.any(1),mask.any(0))].shape[0]
        if (check_shape == 0): # image is too dark so that we crop out everything,
            return img # return original image
        else:
            img1=img[:,:,0][np.ix_(mask.any(1),mask.any(0))]
            img2=img[:,:,1][np.ix_(mask.any(1),mask.any(0))]
            img3=img[:,:,2][np.ix_(mask.any(1),mask.any(0))]
    #         print(img1.shape,img2.shape,img3.shape)
            img = np.stack([img1,img2,img3],axis=-1)
    #         print(img.shape)
        return img

In [0]:
def load_ben_color(image, sigmaX=10):
    #image = cv2.imread(path)
    #image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = crop_image_from_gray(image)
    image = cv2.resize(image, (img_shape, img_shape))
    image=cv2.addWeighted ( image,4, cv2.GaussianBlur( image , (0,0) , sigmaX) ,-4 ,128)
    return image

## Evaluation Metrics

In [0]:
from keras.callbacks import Callback
def get_preds_and_labels(model, generator):
    """
    Get predictions and labels from the generator
    """
    preds = []
    labels = []
    for _ in range(int(np.ceil(generator.samples / BATCH_SIZE))):
        x, y = next(generator)
        preds.append(model.predict(x))
        labels.append(y)
    # Flatten list of numpy arrays
    return np.concatenate(preds).ravel(), np.concatenate(labels).ravel()
# -----------------------------------------------
# -----------------------------------------------
# -----------------------------------------------
class Metrics(Callback):
    """
    A custom Keras callback for saving the best model
    according to the Quadratic Weighted Kappa (QWK) metric
    """
    def on_train_begin(self, logs={}):
        """
        Initialize list of QWK scores on validation data
        """
        self.val_kappas = []

    def on_epoch_end(self, epoch, logs={}):
        """
        Gets QWK score on the validation data
        """
        # Get predictions and convert to integers
        y_pred, labels = get_preds_and_labels(model, val_generator)
        y_pred = np.rint(y_pred).astype(np.uint8).clip(0, 4)
        # We can use sklearns implementation of QWK straight out of the box
        # as long as we specify weights as 'quadratic'
        _val_kappa = cohen_kappa_score(labels, y_pred, weights='quadratic')
        self.val_kappas.append(_val_kappa)
        print(f"val_kappa: {round(_val_kappa, 4)}")
        if _val_kappa == max(self.val_kappas):
            print("Validation Kappa has improved. Saving model.")
            self.model.save(SAVED_MODEL_NAME)
        return
kappa_metrics=Metrics()

Using TensorFlow backend.


## Layers

In [0]:
from keras import backend as K
from keras.layers import Layer
from keras import initializers,regularizers,constraints

class GroupNormalization(Layer):
    """Group normalization layer
    Group Normalization divides the channels into groups and computes within each group
    the mean and variance for normalization. GN's computation is independent of batch sizes,
    and its accuracy is stable in a wide range of batch sizes
    # Arguments
        groups: Integer, the number of groups for Group Normalization.
        axis: Integer, the axis that should be normalized
            (typically the features axis).
            For instance, after a `Conv2D` layer with
            `data_format="channels_first"`,
            set `axis=1` in `BatchNormalization`.
        epsilon: Small float added to variance to avoid dividing by zero.
        center: If True, add offset of `beta` to normalized tensor.
            If False, `beta` is ignored.
        scale: If True, multiply by `gamma`.
            If False, `gamma` is not used.
            When the next layer is linear (also e.g. `nn.relu`),
            this can be disabled since the scaling
            will be done by the next layer.
        beta_initializer: Initializer for the beta weight.
        gamma_initializer: Initializer for the gamma weight.
        beta_regularizer: Optional regularizer for the beta weight.
        gamma_regularizer: Optional regularizer for the gamma weight.
        beta_constraint: Optional constraint for the beta weight.
        gamma_constraint: Optional constraint for the gamma weight.
    # Input shape
        Arbitrary. Use the keyword argument `input_shape`
        (tuple of integers, does not include the samples axis)
        when using this layer as the first layer in a model.
    # Output shape
        Same shape as input.
    # References
        - [Group Normalization](https://arxiv.org/abs/1803.08494)
    """

    def __init__(self,
                 groups=32,
                 axis=-1,
                 epsilon=1e-5,
                 center=True,
                 scale=True,
                 beta_initializer='zeros',
                 gamma_initializer='ones',
                 beta_regularizer=None,
                 gamma_regularizer=None,
                 beta_constraint=None,
                 gamma_constraint=None,
                 **kwargs):
        super(GroupNormalization, self).__init__(**kwargs)
        self.supports_masking = True
        self.groups = groups
        self.axis = axis
        self.epsilon = epsilon
        self.center = center
        self.scale = scale
        self.beta_initializer = initializers.get(beta_initializer)
        self.gamma_initializer = initializers.get(gamma_initializer)
        self.beta_regularizer = regularizers.get(beta_regularizer)
        self.gamma_regularizer = regularizers.get(gamma_regularizer)
        self.beta_constraint = constraints.get(beta_constraint)
        self.gamma_constraint = constraints.get(gamma_constraint)

    def build(self, input_shape):
        dim = input_shape[self.axis]

        if dim is None:
            raise ValueError('Axis ' + str(self.axis) + ' of '
                             'input tensor should have a defined dimension '
                             'but the layer received an input with shape ' +
                             str(input_shape) + '.')

        if dim < self.groups:
            raise ValueError('Number of groups (' + str(self.groups) + ') cannot be '
                             'more than the number of channels (' +
                             str(dim) + ').')

        if dim % self.groups != 0:
            raise ValueError('Number of groups (' + str(self.groups) + ') must be a '
                             'multiple of the number of channels (' +
                             str(dim) + ').')

        self.input_spec = InputSpec(ndim=len(input_shape),
                                    axes={self.axis: dim})
        shape = (dim,)

        if self.scale:
            self.gamma = self.add_weight(shape=shape,
                                         name='gamma',
                                         initializer=self.gamma_initializer,
                                         regularizer=self.gamma_regularizer,
                                         constraint=self.gamma_constraint)
        else:
            self.gamma = None
        if self.center:
            self.beta = self.add_weight(shape=shape,
                                        name='beta',
                                        initializer=self.beta_initializer,
                                        regularizer=self.beta_regularizer,
                                        constraint=self.beta_constraint)
        else:
            self.beta = None
        self.built = True

    def call(self, inputs, **kwargs):
        input_shape = K.int_shape(inputs)
        tensor_input_shape = K.shape(inputs)

        # Prepare broadcasting shape.
        reduction_axes = list(range(len(input_shape)))
        del reduction_axes[self.axis]
        broadcast_shape = [1] * len(input_shape)
        broadcast_shape[self.axis] = input_shape[self.axis] // self.groups
        broadcast_shape.insert(1, self.groups)

        reshape_group_shape = K.shape(inputs)
        group_axes = [reshape_group_shape[i] for i in range(len(input_shape))]
        group_axes[self.axis] = input_shape[self.axis] // self.groups
        group_axes.insert(1, self.groups)

        # reshape inputs to new group shape
        group_shape = [group_axes[0], self.groups] + group_axes[2:]
        group_shape = K.stack(group_shape)
        inputs = K.reshape(inputs, group_shape)

        group_reduction_axes = list(range(len(group_axes)))
        group_reduction_axes = group_reduction_axes[2:]

        mean = K.mean(inputs, axis=group_reduction_axes, keepdims=True)
        variance = K.var(inputs, axis=group_reduction_axes, keepdims=True)

        inputs = (inputs - mean) / (K.sqrt(variance + self.epsilon))

        # prepare broadcast shape
        inputs = K.reshape(inputs, group_shape)
        outputs = inputs

        # In this case we must explicitly broadcast all parameters.
        if self.scale:
            broadcast_gamma = K.reshape(self.gamma, broadcast_shape)
            outputs = outputs * broadcast_gamma

        if self.center:
            broadcast_beta = K.reshape(self.beta, broadcast_shape)
            outputs = outputs + broadcast_beta

        outputs = K.reshape(outputs, tensor_input_shape)

        return outputs

    def get_config(self):
        config = {
            'groups': self.groups,
            'axis': self.axis,
            'epsilon': self.epsilon,
            'center': self.center,
            'scale': self.scale,
            'beta_initializer': initializers.serialize(self.beta_initializer),
            'gamma_initializer': initializers.serialize(self.gamma_initializer),
            'beta_regularizer': regularizers.serialize(self.beta_regularizer),
            'gamma_regularizer': regularizers.serialize(self.gamma_regularizer),
            'beta_constraint': constraints.serialize(self.beta_constraint),
            'gamma_constraint': constraints.serialize(self.gamma_constraint)
        }
        base_config = super(GroupNormalization, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

    def compute_output_shape(self, input_shape):
        return input_shape

# Data Generation

In [0]:
from keras_preprocessing.image import ImageDataGenerator

train_data_gen= ImageDataGenerator(
    horizontal_flip=True,
    #vertical_flip=True,
    rotation_range=60,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=.1,
    rescale=1/255,
    fill_mode='nearest',
    shear_range=0.1,
    #brightness_range=[0.5, 1.5],
    preprocessing_function=load_ben_color)

val_data_gen= ImageDataGenerator(
    horizontal_flip=True,
    #vertical_flip=True,
    rotation_range=60,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=.1,
    rescale=1/255,
    fill_mode='nearest',
    shear_range=0.1,
    #brightness_range=[0.5, 1.5],
    preprocessing_function=load_ben_color)




In [0]:
label_cols=['N','D','G','C','A','H','M','O']
direc='/content/data/ODIR-5K_Training_Dataset/'
img_shape=300
batch=10
train_generator = train_data_gen.flow_from_dataframe(df_train, 
                                                        x_col='Image', 
                                                        y_col=label_cols,
                                                        directory = direc,
                                                        target_size=(img_shape, img_shape),
                                                        batch_size=batch,
                                                        class_mode='other')

val_generator = val_data_gen.flow_from_dataframe(df_validation, 
                                                  x_col='Image', 
                                                  y_col=label_cols,
                                                  directory = direc,
                                                  target_size=(img_shape, img_shape),
                                                  batch_size=batch,
                                                  class_mode='other')

Found 5600 validated image filenames.
Found 1400 validated image filenames.


# Model Setup

In [0]:
from keras.callbacks import ReduceLROnPlateau,ModelCheckpoint,EarlyStopping
SAVED_MODEL_NAME  = 'wieghts.h5'
checkpoint = ModelCheckpoint('model.h5', monitor='val_loss', verbose=1, save_best_only=True, mode='min')
es = EarlyStopping(monitor='val_loss', mode='auto', verbose=1, patience=20)
rlr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, verbose=1,mode='auto')

In [0]:
from efficientnet import EfficientNetB3
effnet = EfficientNetB3(weights='imagenet', include_top=False)

# Replace all Batch Normalization layers by Group Normalization layers
for i, layer in enumerate(effnet.layers):
    if "batch_normalization" in layer.name:
        effnet.layers[i] = GroupNormalization(groups=32, axis=-1, epsilon=0.00001)
        
for i, layer in enumerate(effnet.layers[:-1]):
    effnet.layers[i].train = False













Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
43974656/43966704 [==============================] - 1s 0us/step


In [0]:
from keras.models import Sequential
from keras.layers import GlobalAveragePooling2D,Dropout,Dense
model = Sequential()    
model.add(effnet)
model.add(GlobalAveragePooling2D())
model.add(Dropout(0.5))
model.add(Dense(128, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(8, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['mse', 'acc'])
print(model.summary()) 

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
efficientnet-b3 (Model)      (None, 10, 10, 1536)      10608936  
_________________________________________________________________
global_average_pooling2d_1 ( (None, 1536)              0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 1536)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               196736    
_________________________________________________________________
dense_2 (Dense)              (None, 64)                8256      
_________________________________________

In [0]:
class_weights=[8.77, 8.66, 46.0, 47.,60.0,97.0,57.0,10.0]

model.fit_generator(train_generator,
                    class_weight=class_weights,
                    steps_per_epoch=train_generator.samples // batch,                    
                    epochs=30,
                    validation_data=val_generator,
                    validation_steps = val_generator.samples //batch,                    
                    callbacks=[  rlr,checkpoint]
                    )



Epoch 1/30
560/560 [==============================] - 695s 1s/step - loss: 0.2751 - mean_squared_error: 0.0855 - acc: 0.8794 - val_loss: 0.2609 - val_mean_squared_error: 0.0812 - val_acc: 0.8877

Epoch 00001: val_loss improved from inf to 0.26088, saving model to model.h5
Epoch 2/30
560/560 [==============================] - 649s 1s/step - loss: 0.2519 - mean_squared_error: 0.0787 - acc: 0.8873 - val_loss: 0.2496 - val_mean_squared_error: 0.0787 - val_acc: 0.8863

Epoch 00002: val_loss improved from 0.26088 to 0.24964, saving model to model.h5
Epoch 3/30
560/560 [==============================] - 652s 1s/step - loss: 0.2422 - mean_squared_error: 0.0760 - acc: 0.8911 - val_loss: 0.2350 - val_mean_squared_error: 0.0735 - val_acc: 0.8967

Epoch 00003: val_loss improved from 0.24964 to 0.23496, saving model to model.h5
Epoch 4/30
560/560 [==============================] - 661s 1s/step - loss: 0.2363 - mean_squared_error: 0.0742 - acc: 0.8948 - val_loss: 0.2330 - val_mean_squared_error: 0

In [0]:
model.save_weights(path+"single_model.h5")


In [0]:
# from keras.backend import clear_session as cs
# cs()

# Create Submission File

## SPlit to left and right 

In [0]:
test_sample = pd.read_csv(path+'XYZ_ODIR.csv')
test_sample.head()

,ID,N,D,G,C,A,H,M,O
0,937,0,0,0,0,0,0,0,0
1,967,0,0,0,0,0,0,0,0
2,988,0,0,0,0,0,0,0,0
3,995,0,0,0,0,0,0,0,0
4,1000,0,0,0,0,0,0,0,0


,ID,N,D,G,C,A,H,M,O
0,937,0,0,0,0,0,0,0,0
1,967,0,0,0,0,0,0,0,0
2,988,0,0,0,0,0,0,0,0
3,995,0,0,0,0,0,0,0,0
4,1000,0,0,0,0,0,0,0,0


In [0]:
left_test = pd.DataFrame() 
left_test['ID']     = test_sample.ID
left_test['Image'] = test_sample.ID.apply(lambda x: str(x)+"_left.jpg")

right_test = pd.DataFrame() 
right_test['ID']     = test_sample.ID
right_test['Image'] = test_sample.ID.apply(lambda x: str(x)+"_right.jpg")

print('DF were created.')

In [0]:
left_test.head()

## Data augmentation

In [0]:
test_data_gen= ImageDataGenerator(
                                  rotation_range=360,
                                   horizontal_flip=True,
                                   vertical_flip=True,
                                   validation_split=0.15,
                                   preprocessing_function=load_ben_color, 
                                   rescale=1 / 128.)

In [0]:
test_direc='/content/data/ODIR-5K_Testing_Images'
left_test_generator = test_data_gen.flow_from_dataframe(left_test, 
                                                  x_col='Image', 
                                                  
                                                  directory = test_direc,
                                                  target_size=(img_shape, img_shape),
                                                  batch_size=batch,
                                                  class_mode=None)

right_test_generator = test_data_gen.flow_from_dataframe(right_test, 
                                                  x_col='Image', 
                                                  
                                                  directory = test_direc,
                                                  target_size=(img_shape, img_shape),
                                                  batch_size=batch,
                                                  class_mode=None)

Found 500 validated image filenames.
Found 500 validated image filenames.


## Evaluate

In [0]:
left_pred=model.predict_generator(generator=left_test_generator,steps=left_test_generator.samples//batch)
right_pred=model.predict_generator(generator=right_test_generator,steps=right_test_generator.samples//batch)

In [0]:
left_pred.shape,right_pred.shape

((500, 8), (500, 8))

In [0]:
y_pred= 0.5*left_pred+0.5*right_pred

In [0]:
df_submit=test_sample.copy()
for i,j in enumerate(['N'	,'D'	,'G',	'C',	'A'	,'H'	,'M',	'O']):
  df_submit[j]=y_pred[:,i]

In [0]:
df_submit.head()

,ID,N,D,G,C,A,H,M,O
0,937,0.006293,0.437596,0.000074,0.442128,0.000314,0.000245,0.000068,0.119855
1,967,0.080663,0.006065,0.000402,0.384663,0.420940,0.000706,0.000042,0.108658
2,988,0.605324,0.030372,0.001749,0.001265,0.030142,0.001924,0.000013,0.281375
3,995,0.555466,0.030239,0.009209,0.058077,0.041409,0.004358,0.001286,0.288081
4,1000,0.519771,0.030085,0.001117,0.000171,0.210848,0.002932,0.000157,0.260275


In [0]:
# df_submit[df_submit.iloc[:,1::]>1.0]=1

In [0]:
df_submit.head()

,ID,N,D,G,C,A,H,M,O
0,937,0.006293,0.437596,0.000074,0.442128,0.000314,0.000245,0.000068,0.119855
1,967,0.080663,0.006065,0.000402,0.384663,0.420940,0.000706,0.000042,0.108658
2,988,0.605324,0.030372,0.001749,0.001265,0.030142,0.001924,0.000013,0.281375
3,995,0.555466,0.030239,0.009209,0.058077,0.041409,0.004358,0.001286,0.288081
4,1000,0.519771,0.030085,0.001117,0.000171,0.210848,0.002932,0.000157,0.260275


In [0]:
df_submit.to_csv('TAC_ODIR.csv',index=False)